In [8]:
import json
from tqdm import tqdm

In [9]:
processed_dataset = []
with open("/scratch/lamdo/pyserini_experiments/scirepeval_collections/corpus/scirepeval_search_validation_evaluation.jsonl") as f:
    for line in tqdm(f):
        line = json.loads(line)
        title = line["title"]
        abstract = line["abstract"]
        doc_id = line["doc_id"]

        text = f"{title}. {abstract}"
        text_not_lowered = f"{title}\n{abstract}"
        processed_line = {
            "doc_id": doc_id,
            "text": text,
            "text_not_lowered": text_not_lowered,
            "present_keyphrases": [],
            "absent_keyphrases": []
        }

        processed_dataset.append(processed_line)

630749it [00:12, 48545.00it/s]


In [10]:
sparse_representations = []
with open("/scratch/lamdo/precompute_sparse_representations/custom_trained_combined_references_v8-1--scirepeval_search_validation_evaluation.jsonl") as f:
    for line in f:
        jline = json.loads(line)
        sparse_representations.append(jline)

        if len(sparse_representations) == 20000: break

In [24]:
texts = [item["text"] for item in processed_dataset[:20000]]

In [25]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [26]:
tokenized_texts = [set(tokenizer.tokenize(item)) for item in texts]

Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors


In [27]:
percentages = []
for i in tqdm(range(len(tokenized_texts))):
    total = len(sparse_representations[i])

    num_absent = len([item for item in sparse_representations[i].keys() if item not in tokenized_texts[i]])
    percentages.append(num_absent / total)

100%|██████████| 20000/20000 [00:00<00:00, 58583.71it/s]


In [28]:
import numpy as np

np.mean(percentages)

0.2501171758313665